In [ ]:
pip install pillow

In [ ]:
pip install matplotlib

In [ ]:
pip install numpy

In [ ]:
pip install opencv-python

In [ ]:
conda install cudatoolkit

In [ ]:
conda update -n base -c defaults conda

In [ ]:
conda install -c anaconda tk

# V1

In [ ]:
import numpy as np
from numba import cuda, vectorize, float32
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import matplotlib.pyplot as plt

@cuda.jit
def mandelbrot_kernel(min_x, max_x, min_y, max_y, image, iters):
    height, width = image.shape
    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height

    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y

    for x in range(startX, width, gridX):
        real = min_x + x * pixel_size_x
        for y in range(startY, height, gridY):
            imag = min_y + y * pixel_size_y
            c = complex(real, imag)
            z = 0.0j
            for i in range(iters):
                z = z*z + c
                if (z.real*z.real + z.imag*z.imag) >= 4:
                    image[y, x] = i
                    break
            else:
                image[y, x] = iters - 1

def compute_mandelbrot(min_x, max_x, min_y, max_y, width, height, iters):
    image = np.zeros((height, width), dtype=np.uint8)
    blockdim = (32, 8)
    griddim = (32, 16)

    d_image = cuda.to_device(image)
    mandelbrot_kernel[griddim, blockdim](min_x, max_x, min_y, max_y, d_image, iters)
    d_image.copy_to_host(image)
    return image

def map_colors(image, colormap):
    return colormap(image)

class MandelbrotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Mandelbrot Set Viewer")

        self.zoom_rate = tk.DoubleVar(value=100.0)

        ttk.Label(root, text="Zoom Rate (%)").pack(side=tk.TOP)
        self.zoom_scale = ttk.Scale(root, from_=10, to=500, variable=self.zoom_rate, orient=tk.HORIZONTAL)
        self.zoom_scale.pack(side=tk.TOP, fill=tk.X)

        self.canvas = tk.Canvas(root, width=640, height=480)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.update_button = ttk.Button(root, text="Update View", command=self.update_view)
        self.update_button.pack(side=tk.TOP)

        root.after(100, self.update_view)

    def update_view(self):
        min_x, max_x = -2.0, 1.0
        min_y, max_y = -1.5, 1.5
        width, height = 640, 480
        iters = 256

        image = compute_mandelbrot(min_x, max_x, min_y, max_y, width, height, iters)

        plt_colormap = plt.get_cmap('hot')
        colored_image = map_colors(image, plt_colormap)

        self.create_and_display_image(colored_image)

    def create_and_display_image(self, colored_image):
        pil_image = Image.fromarray(np.uint8(colored_image * 255))
        self.tk_image = ImageTk.PhotoImage(pil_image)

        self.root.update_idletasks()

        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)
        self.canvas.image = self.tk_image

root = tk.Tk()
app = MandelbrotApp(root)
root.mainloop()


# V2

In [ ]:
import numpy as np
from numba import cuda, vectorize, float32
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import matplotlib.pyplot as plt

@cuda.jit
def mandelbrot_kernel(min_x, max_x, min_y, max_y, image, iters):
    height, width = image.shape
    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height

    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y

    for x in range(startX, width, gridX):
        real = min_x + x * pixel_size_x
        for y in range(startY, height, gridY):
            imag = min_y + y * pixel_size_y
            c = complex(real, imag)
            z = 0.0j
            for i in range(iters):
                z = z*z + c
                if (z.real*z.real + z.imag*z.imag) >= 4:
                    image[y, x] = i
                    break
            else:
                image[y, x] = iters - 1

def compute_mandelbrot(min_x, max_x, min_y, max_y, width, height, iters):
    image = np.zeros((height, width), dtype=np.uint8)
    blockdim = (32, 8)
    griddim = (32, 16)

    d_image = cuda.to_device(image)
    mandelbrot_kernel[griddim, blockdim](min_x, max_x, min_y, max_y, d_image, iters)
    d_image.copy_to_host(image)
    return image

def map_colors(image, colormap):
    return colormap(image)

class MandelbrotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Mandelbrot Set Viewer")

        self.zoom_rate = tk.DoubleVar(value=100.0)
        self.real_part = tk.DoubleVar(value=-0.75)
        self.imaginary_part = tk.DoubleVar(value=0.1)
        self.is_zooming = False
        self.zoom_factor = 1.0

        ttk.Label(root, text="Real Part:").pack(side=tk.TOP)
        self.real_entry = ttk.Entry(root, textvariable=self.real_part)
        self.real_entry.pack(side=tk.TOP)

        ttk.Label(root, text="Imaginary Part:").pack(side=tk.TOP)
        self.imag_entry = ttk.Entry(root, textvariable=self.imaginary_part)
        self.imag_entry.pack(side=tk.TOP)

        ttk.Label(root, text="Zoom Rate (%)").pack(side=tk.TOP)
        self.zoom_scale = ttk.Scale(root, from_=10, to=500, variable=self.zoom_rate, orient=tk.HORIZONTAL)
        self.zoom_scale.pack(side=tk.TOP, fill=tk.X)

        self.canvas = tk.Canvas(root, width=640, height=480)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.start_button = ttk.Button(root, text="Start Zoom", command=self.start_zoom)
        self.start_button.pack(side=tk.TOP)
        self.stop_button = ttk.Button(root, text="Stop Zoom", command=self.stop_zoom)
        self.stop_button.pack(side=tk.TOP)

        self.zoom_info_label = ttk.Label(root, text="Zoom: 1x")
        self.zoom_info_label.pack(side=tk.TOP)

        self.update_view()

    def start_zoom(self):
        self.is_zooming = True
        self.perform_zoom()

    def stop_zoom(self):
        self.is_zooming = False

    def perform_zoom(self):
        if self.is_zooming:
            self.zoom_factor *= 1 + (self.zoom_rate.get() / 100.0)
            self.update_view()

            self.root.after(100, self.perform_zoom)

    def update_view(self):
        zoomed_min_x = self.real_part.get() - 2.0 / self.zoom_factor
        zoomed_max_x = self.real_part.get() + 2.0 / self.zoom_factor
        zoomed_min_y = self.imaginary_part.get() - 1.5 / self.zoom_factor
        zoomed_max_y = self.imaginary_part.get() + 1.5 / self.zoom_factor
        width, height = 640, 480
        iters = 256

        image = compute_mandelbrot(zoomed_min_x, zoomed_max_x, zoomed_min_y, zoomed_max_y, width, height, iters)
        colored_image = map_colors(image, plt.get_cmap('hot'))
        self.create_and_display_image(colored_image)

        self.zoom_info_label.config(text=f"Zoom: {self.zoom_factor:.1f}x")

    def create_and_display_image(self, colored_image):
        pil_image = Image.fromarray(np.uint8(colored_image * 255))
        self.tk_image = ImageTk.PhotoImage(pil_image)
        self.root.update_idletasks()
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)
        self.canvas.image = self.tk_image

root = tk.Tk()
app = MandelbrotApp(root)
root.mainloop()


# V3

In [ ]:
import numpy as np
from numba import cuda
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import matplotlib.pyplot as plt

@cuda.jit
def mandelbrot_kernel(min_x, max_x, min_y, max_y, image, iters):
    height, width = image.shape
    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height

    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y

    for x in range(startX, width, gridX):
        real = min_x + x * pixel_size_x
        for y in range(startY, height, gridY):
            imag = min_y + y * pixel_size_y
            c = complex(real, imag)
            z = 0.0j
            for i in range(iters):
                z = z*z + c
                if (z.real*z.real + z.imag*z.imag) >= 4:
                    image[y, x] = i
                    break
            else:
                image[y, x] = iters - 1

def compute_mandelbrot(min_x, max_x, min_y, max_y, width, height, iters):
    image = np.zeros((height, width), dtype=np.uint8)
    blockdim = (32, 8)
    griddim = (32, 16)

    d_image = cuda.to_device(image)
    mandelbrot_kernel[griddim, blockdim](min_x, max_x, min_y, max_y, d_image, iters)
    d_image.copy_to_host(image)
    return image

def map_colors(image, colormap):
    return colormap(image)

class MandelbrotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Mandelbrot Set Viewer")

        default_real = -1.769383179195515018213847286085473782905747263654751437465528216527895619319611191987887653894342701686468599663406934835984123454028053966872660266514595434811793194196363573302709682793695943146714716774002031919065020512659244853735026203310097693791992217030952111815848133482766831104610834983683818312471357751511964121816244219318023485118638666083722365040664254867074683654013015647258306150454564116791483210267781084732430107676079087748369933418914710721955770861925845172959445482333203269525299802359134204528070300441273296202514399300300295833144642522212561596258328029290919911158134003386379705091288237920953499024380913680513871512368411187820615517867601304758195905610284272799932928430918358605799109983971606502824677529924154291921315015305412213104688127397427900716330535760342827838007563501470683376851979682461188231312557396189771618162083272675065421105497587089480320858567022680527745218818472156630048815028139547259136263126046753041127795232821142795073229191744273096953663087561117520098357871045992896473296506796597597442697265155674929747488222847643986370481000300
        default_imaginary = 0.004236847918736772214926507171367997076682670917403757279459435650109714289123299716055036788816829002885173925977803076761048737476905045429119980185020860998018155427488945426274389935307652394018221228924847186786924868561123216151510234503511354417390364732856276244619543076860584832137187844697030796134538976141763739954778083166695249146360254257361839308109726923555954100010797420794587505658052960834305867559444780501963923465355389550069314821112064244320429624763882494974160841648779872028016987521785087924645903329465398276231643914083663624147136110648975955068420983121779008011666212651327286861162395741965312552914549904857397325723486244004135188005807428412957159237798057795264160829345419514069289874005474168888096884296000847011402294633762887085880774518432508597395525887265921365069921205891989833933466150138942740734710689407430631197338592224626876619131697476930710878685450644181128255613118033180069131508494540009670907601269586702849405781191955156272675427890395079218579456719603904005730247062886065173421649366662480064203511248555695897013228991095964254579000000

        self.zoom_rate = tk.DoubleVar(value=100.0)
        self.real_part = tk.DoubleVar(value=default_real)
        self.imaginary_part = tk.DoubleVar(value=default_imaginary)
        self.is_zooming = False
        self.zoom_factor = 1.0

        ttk.Label(root, text="Real Part:").pack(side=tk.TOP)
        self.real_entry = ttk.Entry(root, textvariable=self.real_part)
        self.real_entry.pack(side=tk.TOP)

        ttk.Label(root, text="Imaginary Part:").pack(side=tk.TOP)
        self.imag_entry = ttk.Entry(root, textvariable=self.imaginary_part)
        self.imag_entry.pack(side=tk.TOP)

        ttk.Label(root, text="Zoom Rate (%)").pack(side=tk.TOP)
        self.zoom_scale = ttk.Scale(root, from_=10, to=500, variable=self.zoom_rate, orient=tk.HORIZONTAL)
        self.zoom_scale.pack(side=tk.TOP, fill=tk.X)

        self.canvas = tk.Canvas(root, width=640, height=480)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.start_button = ttk.Button(root, text="Start Zoom", command=self.start_zoom)
        self.start_button.pack(side=tk.TOP)
        self.stop_button = ttk.Button(root, text="Stop Zoom", command=self.stop_zoom)
        self.stop_button.pack(side=tk.TOP)
        self.reset_button = ttk.Button(root, text="Reset Zoom", command=self.reset_zoom)
        self.reset_button.pack(side=tk.TOP)

        self.zoom_info_label = ttk.Label(root, text="Zoom: 1x")
        self.zoom_info_label.pack(side=tk.TOP)

        self.update_view()

    def start_zoom(self):
        self.is_zooming = True
        self.perform_zoom()

    def stop_zoom(self):
        self.is_zooming = False

    def reset_zoom(self):
        self.is_zooming = False
        self.zoom_factor = 1.0
        self.real_part.set(-0.743643887037151)
        self.imaginary_part.set(0.131825904205330)
        self.update_view()

    def perform_zoom(self):
        if self.is_zooming:
            self.zoom_factor *= 1 + (self.zoom_rate.get() / 100.0)
            self.update_view()
            self.root.after(33, self.perform_zoom)  # Faster refresh rate

    def update_view(self):
        zoomed_min_x = self.real_part.get() - 2.0 / self.zoom_factor
        zoomed_max_x = self.real_part.get() + 2.0 / self.zoom_factor
        zoomed_min_y = self.imaginary_part.get() - 1.5 / self.zoom_factor
        zoomed_max_y = self.imaginary_part.get() + 1.5 / self.zoom_factor
        width, height = 640, 480
        iters = 256

        image = compute_mandelbrot(zoomed_min_x, zoomed_max_x, zoomed_min_y, zoomed_max_y, width, height, iters)
        colored_image = map_colors(image, plt.get_cmap('hot'))
        self.create_and_display_image(colored_image)

        self.zoom_info_label.config(text=f"Zoom: {self.zoom_factor:.1f}x")

    def create_and_display_image(self, colored_image):
        pil_image = Image.fromarray(np.uint8(colored_image * 255))
        self.tk_image = ImageTk.PhotoImage(pil_image)
        self.root.update_idletasks()
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)
        self.canvas.image = self.tk_image

root = tk.Tk()
app = MandelbrotApp(root)
root.mainloop()


# V4

In [ ]:
import numpy as np
from numba import cuda
import tkinter as tk
import math
from tkinter import ttk
from PIL import Image, ImageTk

@cuda.jit
def mandelbrot_kernel(min_x, max_x, min_y, max_y, image, iters):
    height, width = image.shape
    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height

    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y

    for x in range(startX, width, gridX):
        real = min_x + x * pixel_size_x
        for y in range(startY, height, gridY):
            imag = min_y + y * pixel_size_y
            c = complex(real, imag)
            z = 0.0j
            for i in range(iters):
                z = z*z + c
                if (z.real*z.real + z.imag*z.imag) >= 4:
                    image[y, x] = i
                    break
            else:
                image[y, x] = iters - 1

def compute_mandelbrot(min_x, max_x, min_y, max_y, width, height, iters):
    image = np.zeros((height, width), dtype=np.uint8)
    blockdim = (32, 8)
    griddim = (32, 16)

    d_image = cuda.to_device(image)
    mandelbrot_kernel[griddim, blockdim](min_x, max_x, min_y, max_y, d_image, iters)
    d_image.copy_to_host(image)
    return image

def map_colors(image, iters):
    colors = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    frequency = 0.1  # Adjust for different color cycles

    for i in range(iters):
        if i == iters - 1:
            # Points inside the Mandelbrot set are black
            colors[image == i] = [0, 0, 0]
        else:
            # Smooth color gradient for points outside the set
            r = int(127.5 * (1 + math.sin(frequency * i + 0)))
            g = int(127.5 * (1 + math.sin(frequency * i + 2 * math.pi / 3)))
            b = int(127.5 * (1 + math.sin(frequency * i + 4 * math.pi / 3)))
            colors[image == i] = [r, g, b]

    return colors

class MandelbrotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Mandelbrot Set Viewer")

        default_real = -1.769383179195515018213847286085473782905747263654751437465528216527895619319611191987887653894342701686468599663406934835984123454028053966872660266514595434811793194196363573302709682793695943146714716774002031919065020512659244853735026203310097693791992217030952111815848133482766831104610834983683818312471357751511964121816244219318023485118638666083722365040664254867074683654013015647258306150454564116791483210267781084732430107676079087748369933418914710721955770861925845172959445482333203269525299802359134204528070300441273296202514399300300295833144642522212561596258328029290919911158134003386379705091288237920953499024380913680513871512368411187820615517867601304758195905610284272799932928430918358605799109983971606502824677529924154291921315015305412213104688127397427900716330535760342827838007563501470683376851979682461188231312557396189771618162083272675065421105497587089480320858567022680527745218818472156630048815028139547259136263126046753041127795232821142795073229191744273096953663087561117520098357871045992896473296506796597597442697265155674929747488222847643986370481000300
        default_imaginary = 0.004236847918736772214926507171367997076682670917403757279459435650109714289123299716055036788816829002885173925977803076761048737476905045429119980185020860998018155427488945426274389935307652394018221228924847186786924868561123216151510234503511354417390364732856276244619543076860584832137187844697030796134538976141763739954778083166695249146360254257361839308109726923555954100010797420794587505658052960834305867559444780501963923465355389550069314821112064244320429624763882494974160841648779872028016987521785087924645903329465398276231643914083663624147136110648975955068420983121779008011666212651327286861162395741965312552914549904857397325723486244004135188005807428412957159237798057795264160829345419514069289874005474168888096884296000847011402294633762887085880774518432508597395525887265921365069921205891989833933466150138942740734710689407430631197338592224626876619131697476930710878685450644181128255613118033180069131508494540009670907601269586702849405781191955156272675427890395079218579456719603904005730247062886065173421649366662480064203511248555695897013228991095964254579000000

        self.zoom_rate = tk.DoubleVar(value=100.0)
        self.real_part = tk.DoubleVar(value=default_real)
        self.imaginary_part = tk.DoubleVar(value=default_imaginary)
        self.is_zooming = False
        self.zoom_factor = 1.0

        ttk.Label(root, text="Real Part:").pack(side=tk.TOP)
        self.real_entry = ttk.Entry(root, textvariable=self.real_part)
        self.real_entry.pack(side=tk.TOP)

        ttk.Label(root, text="Imaginary Part:").pack(side=tk.TOP)
        self.imag_entry = ttk.Entry(root, textvariable=self.imaginary_part)
        self.imag_entry.pack(side=tk.TOP)

        ttk.Label(root, text="Zoom Rate (%)").pack(side=tk.TOP)
        self.zoom_scale = ttk.Scale(root, from_=10, to=500, variable=self.zoom_rate, orient=tk.HORIZONTAL)
        self.zoom_scale.pack(side=tk.TOP, fill=tk.X)

        self.canvas = tk.Canvas(root, width=1920, height=1080)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.start_button = ttk.Button(root, text="Start Zoom", command=self.start_zoom)
        self.start_button.pack(side=tk.TOP)
        self.stop_button = ttk.Button(root, text="Stop Zoom", command=self.stop_zoom)
        self.stop_button.pack(side=tk.TOP)
        self.reset_button = ttk.Button(root, text="Reset Zoom", command=self.reset_zoom)
        self.reset_button.pack(side=tk.TOP)

        self.zoom_info_label = ttk.Label(root, text="Zoom: 1x")
        self.zoom_info_label.pack(side=tk.TOP)

        self.root.bind("<Configure>", self.on_resize)

        self.update_view()

    def on_resize(self, event):
        self.update_view()

    def start_zoom(self):
        self.is_zooming = True
        self.perform_zoom()

    def stop_zoom(self):
        self.is_zooming = False

    def reset_zoom(self):
        self.is_zooming = False
        self.zoom_factor = 1.0
        self.real_part.set(-0.743643887037151)
        self.imaginary_part.set(0.131825904205330)
        self.update_view()

    def perform_zoom(self):
        if self.is_zooming:
            self.zoom_factor *= 1 + (self.zoom_rate.get() / 100.0)
            self.update_view()
            self.root.after(33, self.perform_zoom)  # Faster refresh rate

    def update_view(self):
        width = self.canvas.winfo_width()
        height = self.canvas.winfo_height()

        if width > 1 and height > 1:
            zoomed_min_x = self.real_part.get() - 2.0 / self.zoom_factor
            zoomed_max_x = self.real_part.get() + 2.0 / self.zoom_factor
            zoomed_min_y = self.imaginary_part.get() - 1.5 / self.zoom_factor
            zoomed_max_y = self.imaginary_part.get() + 1.5 / self.zoom_factor

            image = compute_mandelbrot(zoomed_min_x, zoomed_max_x, zoomed_min_y, zoomed_max_y, width, height, 256)
            colored_image = map_colors(image, 256)
            self.create_and_display_image(colored_image)

            self.zoom_info_label.config(text=f"Zoom: {self.zoom_factor:.1f}x")

    def create_and_display_image(self, colored_image):
        pil_image = Image.fromarray(colored_image)
        self.tk_image = ImageTk.PhotoImage(pil_image)
        self.root.update_idletasks()
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)
        self.canvas.image = self.tk_image

root = tk.Tk()
app = MandelbrotApp(root)
root.mainloop()


# V5 - optimized for GTX - 3070 ti 

In [ ]:
import numpy as np
from numba import cuda
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import math

@cuda.jit
def mandelbrot_kernel(min_x, max_x, min_y, max_y, image, iters):
    height, width = image.shape
    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height

    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y

    for x in range(startX, width, gridX):
        real = min_x + x * pixel_size_x
        for y in range(startY, height, gridY):
            imag = min_y + y * pixel_size_y
            z_real, z_imag = 0.0, 0.0
            for i in range(iters):
                z_real_sq, z_imag_sq = z_real*z_real, z_imag*z_imag
                if z_real_sq + z_imag_sq >= 4.0:
                    image[y, x] = i
                    break
                z_imag = 2*z_real*z_imag + imag
                z_real = z_real_sq - z_imag_sq + real
            else:
                image[y, x] = iters - 1

def compute_mandelbrot(min_x, max_x, min_y, max_y, width, height, iters):
    image = np.zeros((height, width), dtype=np.uint8)
    blockdim = (32, 32)
    griddim = (width // blockdim[0] + (width % blockdim[0] > 0),
               height // blockdim[1] + (height % blockdim[1] > 0))

    d_image = cuda.to_device(image)
    mandelbrot_kernel[griddim, blockdim](min_x, max_x, min_y, max_y, d_image, iters)
    d_image.copy_to_host(image)
    return image

def map_colors(image, iters):
    colors = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    frequency = 0.1

    for i in range(iters):
        if i == iters - 1:
            colors[image == i] = [0, 0, 0]
        else:
            r = int(127.5 * (1 + math.sin(frequency * i + 0)))
            g = int(127.5 * (1 + math.sin(frequency * i + 2 * math.pi / 3)))
            b = int(127.5 * (1 + math.sin(frequency * i + 4 * math.pi / 3)))
            colors[image == i] = [r, g, b]

    return colors

class MandelbrotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Mandelbrot Set Viewer")

        # Default interesting location
        default_real = -1.769383179195515018213847286085473782905747263654751437465528216527895619319611191987887653894342701686468599663406934835984123454028053966872660266514595434811793194196363573302709682793695943146714716774002031919065020512659244853735026203310097693791992217030952111815848133482766831104610834983683818312471357751511964121816244219318023485118638666083722365040664254867074683654013015647258306150454564116791483210267781084732430107676079087748369933418914710721955770861925845172959445482333203269525299802359134204528070300441273296202514399300300295833144642522212561596258328029290919911158134003386379705091288237920953499024380913680513871512368411187820615517867601304758195905610284272799932928430918358605799109983971606502824677529924154291921315015305412213104688127397427900716330535760342827838007563501470683376851979682461188231312557396189771618162083272675065421105497587089480320858567022680527745218818472156630048815028139547259136263126046753041127795232821142795073229191744273096953663087561117520098357871045992896473296506796597597442697265155674929747488222847643986370481000300
        default_imaginary = 0.004236847918736772214926507171367997076682670917403757279459435650109714289123299716055036788816829002885173925977803076761048737476905045429119980185020860998018155427488945426274389935307652394018221228924847186786924868561123216151510234503511354417390364732856276244619543076860584832137187844697030796134538976141763739954778083166695249146360254257361839308109726923555954100010797420794587505658052960834305867559444780501963923465355389550069314821112064244320429624763882494974160841648779872028016987521785087924645903329465398276231643914083663624147136110648975955068420983121779008011666212651327286861162395741965312552914549904857397325723486244004135188005807428412957159237798057795264160829345419514069289874005474168888096884296000847011402294633762887085880774518432508597395525887265921365069921205891989833933466150138942740734710689407430631197338592224626876619131697476930710878685450644181128255613118033180069131508494540009670907601269586702849405781191955156272675427890395079218579456719603904005730247062886065173421649366662480064203511248555695897013228991095964254579000000

        self.zoom_rate = tk.DoubleVar(value=100.0)
        self.real_part = tk.DoubleVar(value=default_real)
        self.imaginary_part = tk.DoubleVar(value=default_imaginary)
        self.is_zooming = False
        self.zoom_factor = 1.0

        ttk.Label(root, text="Real Part:").pack(side=tk.TOP)
        self.real_entry = ttk.Entry(root, textvariable=self.real_part)
        self.real_entry.pack(side=tk.TOP)

        ttk.Label(root, text="Imaginary Part:").pack(side=tk.TOP)
        self.imag_entry = ttk.Entry(root, textvariable=self.imaginary_part)
        self.imag_entry.pack(side=tk.TOP)

        ttk.Label(root, text="Zoom Rate (%)").pack(side=tk.TOP)
        self.zoom_scale = ttk.Scale(root, from_=10, to=500, variable=self.zoom_rate, orient=tk.HORIZONTAL)
        self.zoom_scale.pack(side=tk.TOP, fill=tk.X)

        self.canvas = tk.Canvas(root, width=1920, height=1080)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.start_button = ttk.Button(root, text="Start Zoom", command=self.start_zoom)
        self.start_button.pack(side=tk.TOP)
        self.stop_button = ttk.Button(root, text="Stop Zoom", command=self.stop_zoom)
        self.stop_button.pack(side=tk.TOP)
        self.reset_button = ttk.Button(root, text="Reset Zoom", command=self.reset_zoom)
        self.reset_button.pack(side=tk.TOP)

        self.zoom_info_label = ttk.Label(root, text="Zoom: 1x")
        self.zoom_info_label.pack(side=tk.TOP)

        self.root.bind("<Configure>", self.on_resize)

        self.update_view()

    def on_resize(self, event):
        self.update_view()

    def start_zoom(self):
        self.is_zooming = True
        self.perform_zoom()

    def stop_zoom(self):
        self.is_zooming = False

    def reset_zoom(self):
        self.is_zooming = False
        self.zoom_factor = 1.0
        self.real_part.set(-0.743643887037151)
        self.imaginary_part.set(0.131825904205330)
        self.update_view()

    def perform_zoom(self):
        if self.is_zooming:
            self.zoom_factor *= 1 + (self.zoom_rate.get() / 100.0)
            self.update_view()
            self.root.after(33, self.perform_zoom)  # Faster refresh rate

    def update_view(self):
        width = self.canvas.winfo_width()
        height = self.canvas.winfo_height()

        if width > 1 and height > 1:
            zoomed_min_x = self.real_part.get() - 2.0 / self.zoom_factor
            zoomed_max_x = self.real_part.get() + 2.0 / self.zoom_factor
            zoomed_min_y = self.imaginary_part.get() - 1.5 / self.zoom_factor
            zoomed_max_y = self.imaginary_part.get() + 1.5 / self.zoom_factor

            image = compute_mandelbrot(zoomed_min_x, zoomed_max_x, zoomed_min_y, zoomed_max_y, width, height, 256)
            colored_image = map_colors(image, 256)
            self.create_and_display_image(colored_image)

            self.zoom_info_label.config(text=f"Zoom: {self.zoom_factor:.1f}x")

    def create_and_display_image(self, colored_image):
        pil_image = Image.fromarray(colored_image)
        self.tk_image = ImageTk.PhotoImage(pil_image)
        self.root.update_idletasks()
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)
        self.canvas.image = self.tk_image

root = tk.Tk()
app = MandelbrotApp(root)
root.mainloop()


# with video creation

In [2]:
import numpy as np
from numba import cuda
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk
import math
import cv2
import os

@cuda.jit
def mandelbrot_kernel(min_x, max_x, min_y, max_y, image, iters):
    height, width = image.shape
    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height

    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y

    for x in range(startX, width, gridX):
        real = min_x + x * pixel_size_x
        for y in range(startY, height, gridY):
            imag = min_y + y * pixel_size_y
            z_real, z_imag = 0.0, 0.0
            for i in range(iters):
                z_real_sq, z_imag_sq = z_real*z_real, z_imag*z_imag
                if z_real_sq + z_imag_sq >= 4.0:
                    image[y, x] = i
                    break
                z_imag = 2*z_real*z_imag + imag
                z_real = z_real_sq - z_imag_sq + real
            else:
                image[y, x] = iters - 1

def compute_mandelbrot(min_x, max_x, min_y, max_y, width, height, iters):
    image = np.zeros((height, width), dtype=np.uint8)
    blockdim = (32, 32)
    griddim = (width // blockdim[0] + (width % blockdim[0] > 0),
               height // blockdim[1] + (height % blockdim[1] > 0))

    d_image = cuda.to_device(image)
    mandelbrot_kernel[griddim, blockdim](min_x, max_x, min_y, max_y, d_image, iters)
    d_image.copy_to_host(image)
    return image

def map_colors(image, iters):
    colors = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    frequency = 0.1

    for i in range(iters):
        if i == iters - 1:
            colors[image == i] = [0, 0, 0]
        else:
            r = int(127.5 * (1 + math.sin(frequency * i + 0)))
            g = int(127.5 * (1 + math.sin(frequency * i + 2 * math.pi / 3)))
            b = int(127.5 * (1 + math.sin(frequency * i + 4 * math.pi / 3)))
            colors[image == i] = [r, g, b]

    return colors

class MandelbrotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Mandelbrot Set Viewer")

        default_real = -1.769383179195515018213847286085473782905747263654751437465528216527895619319611191987887653894342701686468599663406934835984123454028053966872660266514595434811793194196363573302709682793695943146714716774002031919065020512659244853735026203310097693791992217030952111815848133482766831104610834983683818312471357751511964121816244219318023485118638666083722365040664254867074683654013015647258306150454564116791483210267781084732430107676079087748369933418914710721955770861925845172959445482333203269525299802359134204528070300441273296202514399300300295833144642522212561596258328029290919911158134003386379705091288237920953499024380913680513871512368411187820615517867601304758195905610284272799932928430918358605799109983971606502824677529924154291921315015305412213104688127397427900716330535760342827838007563501470683376851979682461188231312557396189771618162083272675065421105497587089480320858567022680527745218818472156630048815028139547259136263126046753041127795232821142795073229191744273096953663087561117520098357871045992896473296506796597597442697265155674929747488222847643986370481000300
        default_imaginary = 0.004236847918736772214926507171367997076682670917403757279459435650109714289123299716055036788816829002885173925977803076761048737476905045429119980185020860998018155427488945426274389935307652394018221228924847186786924868561123216151510234503511354417390364732856276244619543076860584832137187844697030796134538976141763739954778083166695249146360254257361839308109726923555954100010797420794587505658052960834305867559444780501963923465355389550069314821112064244320429624763882494974160841648779872028016987521785087924645903329465398276231643914083663624147136110648975955068420983121779008011666212651327286861162395741965312552914549904857397325723486244004135188005807428412957159237798057795264160829345419514069289874005474168888096884296000847011402294633762887085880774518432508597395525887265921365069921205891989833933466150138942740734710689407430631197338592224626876619131697476930710878685450644181128255613118033180069131508494540009670907601269586702849405781191955156272675427890395079218579456719603904005730247062886065173421649366662480064203511248555695897013228991095964254579000000

        self.zoom_rate = tk.DoubleVar(value=100.0)
        self.real_part = tk.DoubleVar(value=default_real)
        self.imaginary_part = tk.DoubleVar(value=default_imaginary)
        self.is_zooming = False
        self.zoom_factor = 1.0

        ttk.Label(root, text="Real Part:").pack(side=tk.TOP)
        self.real_entry = ttk.Entry(root, textvariable=self.real_part)
        self.real_entry.pack(side=tk.TOP)

        ttk.Label(root, text="Imaginary Part:").pack(side=tk.TOP)
        self.imag_entry = ttk.Entry(root, textvariable=self.imaginary_part)
        self.imag_entry.pack(side=tk.TOP)

        ttk.Label(root, text="Zoom Rate (%)").pack(side=tk.TOP)
        self.zoom_scale = ttk.Scale(root, from_=10, to=500, variable=self.zoom_rate, orient=tk.HORIZONTAL)
        self.zoom_scale.pack(side=tk.TOP, fill=tk.X)

        self.canvas = tk.Canvas(root, width=1920, height=1080)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.start_button = ttk.Button(root, text="Start Zoom", command=self.start_zoom)
        self.start_button.pack(side=tk.TOP)
        self.stop_button = ttk.Button(root, text="Stop Zoom", command=self.stop_zoom)
        self.stop_button.pack(side=tk.TOP)
        self.reset_button = ttk.Button(root, text="Reset Zoom", command=self.reset_zoom)
        self.reset_button.pack(side=tk.TOP)

        self.zoom_info_label = ttk.Label(root, text="Zoom: 1x")
        self.zoom_info_label.pack(side=tk.TOP)

        self.root.bind("<Configure>", self.on_resize)

        self.is_recording = False
        self.frames = []

        self.update_view()

    def on_resize(self, event):
        self.update_view()

    def start_zoom(self):
        if not self.is_recording:
            self.is_recording = True
            self.frames = []
            messagebox.showinfo("Recording Started", "Recording started. Press 'Stop Zoom' to save the video.")
            self.perform_zoom()

    def stop_zoom(self):
        if self.is_recording:
            self.is_recording = False
            if self.frames:
                self.save_video()
                messagebox.showinfo("Recording Stopped", "Video saved as 'mandelbrot.mp4'.")

    def perform_zoom(self):
        if self.is_recording:
            self.zoom_factor *= 1 + (self.zoom_rate.get() / 100.0)
            self.update_view()
            self.root.after(33, self.perform_zoom)

    def reset_zoom(self):
        self.is_recording = False
        self.zoom_factor = 1.0
        self.real_part.set(-0.743643887037151)
        self.imaginary_part.set(0.131825904205330)
        self.update_view()

    def update_view(self):
        width = self.canvas.winfo_width()
        height = self.canvas.winfo_height()

        if width > 1 and height > 1:
            zoomed_min_x = self.real_part.get() - 2.0 / self.zoom_factor
            zoomed_max_x = self.real_part.get() + 2.0 / self.zoom_factor
            zoomed_min_y = self.imaginary_part.get() - 1.5 / self.zoom_factor
            zoomed_max_y = self.imaginary_part.get() + 1.5 / self.zoom_factor

            image = compute_mandelbrot(zoomed_min_x, zoomed_max_x, zoomed_min_y, zoomed_max_y, width, height, 256)
            colored_image = map_colors(image, 256)
            self.create_and_display_image(colored_image)

            self.zoom_info_label.config(text=f"Zoom: {self.zoom_factor:.1f}x")

        if self.is_recording:
            bgr_image = cv2.cvtColor(colored_image, cv2.COLOR_RGB2BGR)  
            self.frames.append(bgr_image)

    def create_and_display_image(self, colored_image):
        pil_image = Image.fromarray(colored_image)
        self.tk_image = ImageTk.PhotoImage(pil_image)
        self.root.update_idletasks()
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)
        self.canvas.image = self.tk_image

    def save_video(self):
        try:
            base_dir = os.path.dirname(__file__)
        except NameError:
            base_dir = os.path.expanduser("~")  
    
        output_file = os.path.join(base_dir, "mandelbrot.mp4")  
        if self.frames:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            height, width, layers = self.frames[0].shape
            video_writer = cv2.VideoWriter(output_file, fourcc, 30, (width, height))
    
            for frame in self.frames:
                video_writer.write(frame)
    
            video_writer.release()
            self.frames = []  
        else:
            print("No frames to save")

if __name__ == "__main__":
    root = tk.Tk()
    app = MandelbrotApp(root)
    root.mainloop()


# Add Later: more precition for more zoom